In [30]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import torch
import torch.nn as nn
import torch.optim as optim

In [31]:
# Load your data
df = pd.read_csv('../data/10000_games_2023-08-07 19:12:18.727623.csv')

# Select columns you want to use for prediction
selected_columns = ['dealt', 'first_to_play', 'dealer', 'wild_suit',
                    'player0desired', 'player1desired', 'player2desired', 'player3desired',
                    'hand1', 'hand2', 'hand3', 'hand4', 'hand5', 'hand6', 'hand7',
                    'hand8', 'hand9', 'deciding_player']

X = df[selected_columns]
y = df['desired']

# Normalize data
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Split into training and testing datasets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert data to PyTorch tensors
X_train = torch.FloatTensor(X_train)
y_train = torch.FloatTensor(y_train.to_numpy())
X_test = torch.FloatTensor(X_test)
y_test = torch.FloatTensor(y_test.to_numpy())

FileNotFoundError: [Errno 2] No such file or directory: '../data/10000_games_2023-08-07 19:12:18.727623.csv.csv'

In [24]:
df

,dealt,desired,first_to_play,dealer,wild_suit,player0desired,player1desired,player2desired,player3desired,result,hand1,hand2,hand3,hand4,hand5,hand6,hand7,hand8,hand9,deciding_player
0,1,1,3,2,1,-1,-1,-1,0,1,34,44,44,44,44,44,44,44,44,0
1,1,0,3,2,1,1,-1,-1,0,1,30,44,44,44,44,44,44,44,44,1
2,1,0,3,2,1,1,0,-1,0,1,16,44,44,44,44,44,44,44,44,2
3,1,0,3,2,1,-1,-1,-1,-1,1,17,44,44,44,44,44,44,44,44,3
4,2,0,0,3,3,-1,-1,-1,-1,1,3,20,44,44,44,44,44,44,44,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
591610,1,0,3,1,3,-1,-1,-1,-1,1,1,44,44,44,44,44,44,44,44,3
591611,9,1,1,3,3,-1,-1,-1,-1,1,31,19,16,27,0,10,11,34,26,1
591612,9,3,0,0,3,2,-1,-1,-1,1,22,34,20,21,18,12,13,16,25,1
591613,9,1,0,0,3,2,3,-1,-1,1,11,6,15,24,10,1,0,19,9,2


In [26]:
# Define your model
class Net(nn.Module):
    def __init__(self, n_features):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(n_features, 10)
        self.fc2 = nn.Linear(10, 10)
        self.fc3 = nn.Linear(10, 10)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.log_softmax(self.fc3(x), dim=1)
        return x

# Initialize the model and define loss and optimizer
model = Net(X_train.shape[1])
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

In [27]:
# Convert your labels to long datatype
y_train = y_train.long()
y_test = y_test.long()
epochs = 1000

# Train the model
for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()
    out = model(X_train)
    loss = criterion(out, y_train)
    loss.backward()
    optimizer.step()
    if epoch % 10 == 0:
        print('Train Epoch: {} \tLoss: {:.6f}'.format(epoch, loss.item()))

# Evaluate the model
model.eval()
with torch.no_grad():
    output = model(X_test)
    predicted = torch.argmax(output, dim=1)
    correct = (predicted == y_test).sum().item()
    accuracy = correct / len(y_test)
    print('Test accuracy: {:.2f}%'.format(accuracy * 100))

Train Epoch: 0 	Loss: 2.489653
Train Epoch: 10 	Loss: 2.441167
Train Epoch: 20 	Loss: 2.395143
Train Epoch: 30 	Loss: 2.351152
Train Epoch: 40 	Loss: 2.308835
Train Epoch: 50 	Loss: 2.267894
Train Epoch: 60 	Loss: 2.228070
Train Epoch: 70 	Loss: 2.189169
Train Epoch: 80 	Loss: 2.151021
Train Epoch: 90 	Loss: 2.113489
Train Epoch: 100 	Loss: 2.076461
Train Epoch: 110 	Loss: 2.039850
Train Epoch: 120 	Loss: 2.003589
Train Epoch: 130 	Loss: 1.967628
Train Epoch: 140 	Loss: 1.931931
Train Epoch: 150 	Loss: 1.896480
Train Epoch: 160 	Loss: 1.861276
Train Epoch: 170 	Loss: 1.826334
Train Epoch: 180 	Loss: 1.791685
Train Epoch: 190 	Loss: 1.757374
Train Epoch: 200 	Loss: 1.723448
Train Epoch: 210 	Loss: 1.689964
Train Epoch: 220 	Loss: 1.656978
Train Epoch: 230 	Loss: 1.624539
Train Epoch: 240 	Loss: 1.592701
Train Epoch: 250 	Loss: 1.561504
Train Epoch: 260 	Loss: 1.530971
Train Epoch: 270 	Loss: 1.501124
Train Epoch: 280 	Loss: 1.471979
Train Epoch: 290 	Loss: 1.443531
Train Epoch: 300 	Los

In [28]:
from datetime import datetime
torch.save(model.state_dict(), '../models/calling_model' + str(epochs) + '_epochs_' + str(datetime.now()) + '.pth')

In [29]:
from sklearn.preprocessing import StandardScaler
from joblib import dump
# Now that your scaler is fitted, you can save it
dump(scaler, '../models/' + str(datetime.now()) + 'scaler.joblib') 

['../models/2023-08-07 18:17:05.839020scaler.joblib']

In [9]:
from models.model_structure_1 import Net
# Initialize the model with the same architecture
model = Net(X_train.shape[1])  # replace n_features with the number of input features

# Load the state dictionary
model.load_state_dict(torch.load('../models/calling_model10_epochs_2023-08-07 16:01:19.990395.pth'))

# Make sure to call model.eval() method before inferencing to set the dropout and batch normalization layers to evaluation mode
model.eval()
with torch.no_grad():
    output = model(X_test)
    predicted = torch.argmax(output, dim=1)
    correct = (predicted == y_test).sum().item()
    accuracy = correct / len(y_test)
    print('Test accuracy: {:.2f}%'.format(accuracy * 100))

Test accuracy: 16.84%


In [10]:
X_train.shape[1]

18